This notebook is for playing around with the Weighted Reservoir Sampling irrespective of complex images.

In [6]:
from typing import Dict, List, Tuple

from src.reservoir import Reservoir


In [ ]:
def create_element_stream(num_elements: int) -> List[Tuple[float, float]]:
	stream = []


	return stream

SyntaxError: incomplete input (3277757782.py, line 1)

In [ ]:
reservoir = Reservoir()




SyntaxError: invalid syntax (1977235869.py, line 1)